<a href="https://colab.research.google.com/github/Fernsrea/flow_matching/blob/main/CIFAR_10_Discreet_Reduced_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!git clone https://github.com/Fernsrea/flow_matching.git
%cd flow_matching

Cloning into 'flow_matching'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 246 (delta 41), reused 21 (delta 21), pack-reused 165 (from 1)
Receiving objects: 100% (246/246), 3.21 MiB | 32.83 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/flow_matching/examples/image/flow_matching/examples/image/flow_matching


In [14]:
%cd examples/image

/content/flow_matching/examples/image/flow_matching/examples/image/flow_matching/examples/image


In [15]:
!pip install -r requirements.txt

In [16]:
!pip install torchdiffeq

In [17]:
!pip install flow_matching

In [ ]:
!python train.py \
--dataset=cifar10 \
--batch_size=64 \
--accum_iter=1 \
--eval_frequency=100 \
--epochs=3000 \
--use_ema \
--discrete_flow_matching

Not using distributed mode
2025-05-01 18:18:21 INFO     job dir: /content/flow_matching/examples/image/flow_matching/examples/image/flow_matching/examples/image
2025-05-01 18:18:21 INFO     Namespace(batch_size=64,
epochs=3000,
accum_iter=1,
lr=0.0001,
optimizer_betas=[0.9,
0.95],
decay_lr=False,
class_drop_prob=0.2,
skewed_timesteps=False,
edm_schedule=False,
use_ema=True,
dataset='cifar10',
data_path='./data/image_generation',
output_dir='./output_dir',
ode_method='midpoint',
ode_options={'step_size': 0.01},
sym=0.0,
temp=1.0,
sym_func=False,
sampling_dtype='float32',
cfg_scale=0.2,
fid_samples=50000,
device='cuda',
seed=0,
resume='',
start_epoch=0,
eval_only=False,
eval_frequency=100,
compute_fid=False,
save_fid_samples=False,
num_workers=10,
pin_mem=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
dist_url='env://',
test_run=False,
discrete_flow_matching=True,
discrete_fm_steps=1024,
distributed=False)
2025-05-01 18:18:21 INFO     Saving args to output_dir/args.json
2025-05-01

In [ ]:
!python train.py --dataset cifar10 --output_dir output_cifar10 --batch_size 64 --accum_iter=1 --eval_frequency=100 --epochs=3000 --class_drop_prob=1.0 --cfg_scale=0.0 --compute_fid --ode_method heun2 --ode_options '{"nfe": 50}' --use_ema --edm_schedule --skewed_timesteps

Not using distributed mode
2025-04-17 08:53:07 INFO     job dir: /content/flow_matching/examples/image
2025-04-17 08:53:07 INFO     Namespace(batch_size=64,
epochs=3000,
accum_iter=1,
lr=0.0001,
optimizer_betas=[0.9,
0.95],
decay_lr=False,
class_drop_prob=1.0,
skewed_timesteps=True,
edm_schedule=True,
use_ema=True,
dataset='cifar10',
data_path='./data/image_generation',
output_dir='output_cifar10',
ode_method='heun2',
ode_options={'nfe': 50},
sym=0.0,
temp=1.0,
sym_func=False,
sampling_dtype='float32',
cfg_scale=0.0,
fid_samples=50000,
device='cuda',
seed=0,
resume='',
start_epoch=0,
eval_only=False,
eval_frequency=100,
compute_fid=True,
save_fid_samples=False,
num_workers=10,
pin_mem=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
dist_url='env://',
test_run=False,
discrete_flow_matching=False,
discrete_fm_steps=1024,
distributed=False)
2025-04-17 08:53:07 INFO     Saving args to output_cifar10/args.json
2025-04-17 08:53:07 INFO     Initializing Dataset: cifar10
100% 170M/170M [

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir output_cifar10


In [ ]:
!pip install submitit

In [ ]:
import os
from pathlib import Path
import uuid  # Import the uuid module (although it's already in your submitit_train.py)

shared_dir = "/content/shared_experiments" # Choose a path within /content/

if not Path(shared_dir).is_dir():
    os.makedirs(shared_dir, exist_ok=True)

def get_shared_folder(shared_dir: str) -> Path:
    user = os.getenv("USER", "default_user") # Colab might not always have USER set
    p = Path(shared_dir) / user / "experiments"
    p.mkdir(parents=True, exist_ok=True)
    return p

def get_init_file(shared_dir: str):
    shared_folder = get_shared_folder(shared_dir)
    init_file = shared_folder / f"{uuid.uuid4().hex}_init"
    if init_file.exists():
        os.remove(str(init_file))
    return init_file

# You can optionally run these lines to see the paths that will be used
job_dir = get_shared_folder(shared_dir) / "my_job" # Example job directory
print(f"Job directory: {job_dir}")
init_file = get_init_file(shared_dir)
print(f"Init file: {init_file}")

Job directory: /content/shared_experiments/default_user/experiments/my_job
Init file: /content/shared_experiments/default_user/experiments/af6edabdc1e248e18abdf42794a7eaf6_init


In [ ]:
import os

os.environ['USER'] = 'thesis_dfki'

In [ ]:
!python submitit_train.py \
    --dataset=cifar10 \
    --batch_size=64 \
    --nodes=1 \
    --accum_iter=1 \
    --eval_frequency=100 \
    --epochs=3000 \
    --class_drop_prob=1.0 \
    --cfg_scale=0.0 \
    --compute_fid \
    --ode_method heun2 \
    --ode_options '{"nfe": 50}' \
    --use_ema \
    --edm_schedule \
    --skewed_timesteps \
    --output_dir output_cifar10_submitit \
    --shared_dir "/content/shared_experiments"

2025-04-21 18:40:36 INFO     Submitted job 5326


In [ ]:
from pathlib import Path
import json

args = {
    "output_dir": Path("/tmp/some/path"),
    "other_param": 42,
}

with open("args.json", "w") as f:
    json.dump({k: str(v) if isinstance(v, Path) else v for k, v in args.items()}, f)
